In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
df = pd.read_csv('estat_une_rt_m.csv')
df.columns = df.columns.str.strip()
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df.head()

,freq,s_adj,age,unit,sex,geo,1983-01,1983-02,1983-03,1983-04,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,M,NSA,TOTAL,PC_ACT,F,AT,:,:,:,:,...,5.0,4.8,4.2,4.6,4.3,4.6,5.2,5.6,4.2,:
1,M,NSA,TOTAL,PC_ACT,F,BA,:,:,:,:,...,17.3,16.5,16.3,16.3,16.1,16.1,15.9,15.7,:,:
2,M,NSA,TOTAL,PC_ACT,F,BE,:,:,:,:,...,4.9,4.9,4.9,5.0,5.6,5.8,5.8,5.5,5.2,:
3,M,NSA,TOTAL,PC_ACT,F,BG,:,:,:,:,...,4.9,4.5,4.0,3.7,3.5,3.3,3.3,3.5,3.5,:
4,M,NSA,TOTAL,PC_ACT,F,CH,:,:,:,:,...,4.4,4.2,4.2,4.3,4.8,5.1,5.2,:,:,:


In [3]:
path = '../ESTAT_CODELISTS/*.tsv'
files = glob.glob(path)
dataframes = {file.split('/')[-1].replace('.tsv', ''): pd.read_csv(
    file, sep='\t', on_bad_lines='skip') for file in files}

In [4]:
for col in ["freq", "unit", "sex", "age", "s_adj", "geo"]:
    curr_codelist = dataframes[f"ESTAT_{col.upper()}"]
    df[col] = df[col].map(curr_codelist.set_index('CODE')['Label'])

df.rename(columns={'geo': 'country'}, inplace=True)
df.head()

,freq,s_adj,age,unit,sex,country,1983-01,1983-02,1983-03,1983-04,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Monthly,Unadjusted data (i.e. neither seasonally adjus...,Total,Percentage of population in the labour force,Females,Austria,:,:,:,:,...,5.0,4.8,4.2,4.6,4.3,4.6,5.2,5.6,4.2,:
1,Monthly,Unadjusted data (i.e. neither seasonally adjus...,Total,Percentage of population in the labour force,Females,Bosnia and Herzegovina,:,:,:,:,...,17.3,16.5,16.3,16.3,16.1,16.1,15.9,15.7,:,:
2,Monthly,Unadjusted data (i.e. neither seasonally adjus...,Total,Percentage of population in the labour force,Females,Belgium,:,:,:,:,...,4.9,4.9,4.9,5.0,5.6,5.8,5.8,5.5,5.2,:
3,Monthly,Unadjusted data (i.e. neither seasonally adjus...,Total,Percentage of population in the labour force,Females,Bulgaria,:,:,:,:,...,4.9,4.5,4.0,3.7,3.5,3.3,3.3,3.5,3.5,:
4,Monthly,Unadjusted data (i.e. neither seasonally adjus...,Total,Percentage of population in the labour force,Females,Switzerland,:,:,:,:,...,4.4,4.2,4.2,4.3,4.8,5.1,5.2,:,:,:


In [5]:
df.replace(": z", np.nan, inplace=True)
df.replace(":", np.nan, inplace=True)

In [6]:
year_cols = [col for col in df.columns if col.isdigit()]

for col in year_cols:
    df[col] = df[col].astype(str).str.extract(r'(\d+)').astype(float)

In [7]:
df.head()

,freq,s_adj,age,unit,sex,country,1983-01,1983-02,1983-03,1983-04,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Monthly,Unadjusted data (i.e. neither seasonally adjus...,Total,Percentage of population in the labour force,Females,Austria,NaN,NaN,NaN,NaN,...,5.0,4.8,4.2,4.6,4.3,4.6,5.2,5.6,4.2,NaN
1,Monthly,Unadjusted data (i.e. neither seasonally adjus...,Total,Percentage of population in the labour force,Females,Bosnia and Herzegovina,NaN,NaN,NaN,NaN,...,17.3,16.5,16.3,16.3,16.1,16.1,15.9,15.7,NaN,NaN
2,Monthly,Unadjusted data (i.e. neither seasonally adjus...,Total,Percentage of population in the labour force,Females,Belgium,NaN,NaN,NaN,NaN,...,4.9,4.9,4.9,5.0,5.6,5.8,5.8,5.5,5.2,NaN
3,Monthly,Unadjusted data (i.e. neither seasonally adjus...,Total,Percentage of population in the labour force,Females,Bulgaria,NaN,NaN,NaN,NaN,...,4.9,4.5,4.0,3.7,3.5,3.3,3.3,3.5,3.5,NaN
4,Monthly,Unadjusted data (i.e. neither seasonally adjus...,Total,Percentage of population in the labour force,Females,Switzerland,NaN,NaN,NaN,NaN,...,4.4,4.2,4.2,4.3,4.8,5.1,5.2,NaN,NaN,NaN


In [8]:
if df["freq"].nunique() == 1:
    df.drop(columns=["freq"], inplace=True)
if df["unit"].nunique() == 1:
    df.drop(columns=["unit"], inplace=True)

In [9]:
df.head()

,s_adj,age,unit,sex,country,1983-01,1983-02,1983-03,1983-04,1983-05,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Unadjusted data (i.e. neither seasonally adjus...,Total,Percentage of population in the labour force,Females,Austria,NaN,NaN,NaN,NaN,NaN,...,5.0,4.8,4.2,4.6,4.3,4.6,5.2,5.6,4.2,NaN
1,Unadjusted data (i.e. neither seasonally adjus...,Total,Percentage of population in the labour force,Females,Bosnia and Herzegovina,NaN,NaN,NaN,NaN,NaN,...,17.3,16.5,16.3,16.3,16.1,16.1,15.9,15.7,NaN,NaN
2,Unadjusted data (i.e. neither seasonally adjus...,Total,Percentage of population in the labour force,Females,Belgium,NaN,NaN,NaN,NaN,NaN,...,4.9,4.9,4.9,5.0,5.6,5.8,5.8,5.5,5.2,NaN
3,Unadjusted data (i.e. neither seasonally adjus...,Total,Percentage of population in the labour force,Females,Bulgaria,NaN,NaN,NaN,NaN,NaN,...,4.9,4.5,4.0,3.7,3.5,3.3,3.3,3.5,3.5,NaN
4,Unadjusted data (i.e. neither seasonally adjus...,Total,Percentage of population in the labour force,Females,Switzerland,NaN,NaN,NaN,NaN,NaN,...,4.4,4.2,4.2,4.3,4.8,5.1,5.2,NaN,NaN,NaN


In [10]:
df.to_csv("./clean/estat_une_rt_m.csv", index=False)